In [2]:
# Import of the required libraries
import georinex as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import datetime as dtt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

# Import of the software's modules
import functions as fn
import transformations as trf
import pointPositioning as pp
import RINEXreader 

In [3]:
param_file = 'parameters.txt'
file = open(param_file, 'r')
p = file.read().split('\n')

# Definition of the paths to the RINEX files
nav_path = p[0]
obs_path = p[1]
eph_path = p[2]
cutoff = int(p[3])

# Load with georinex library of nav and obs rinex info

In [6]:
# Loading of GPS navigation parameters from NAV RINEX
param_gps = gr.load(nav_path)
df_gps = param_gps.to_dataframe()
df_gps = ((df_gps.dropna()).sort_values(by=['time'], ascending = True)).reset_index()
# Corrupted timestamps are corrected
df_gps = fn.fixTime(df_gps)

# Loading of GPS code observations in a DataSet
obs = gr.load(obs_path)

data = df_gps['time'][0]

C:\git\pointPositioning\pointPositioning\functions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'][i] = t


In [7]:
print(obs.to_dataframe())

                                   C1            L1    S1            P1  \
sv  time                                                                  
G01 2021-05-26 00:00:00           NaN           NaN   NaN           NaN   
    2021-05-26 00:00:30           NaN           NaN   NaN           NaN   
    2021-05-26 00:01:00           NaN           NaN   NaN           NaN   
    2021-05-26 00:01:30           NaN           NaN   NaN           NaN   
    2021-05-26 00:02:00           NaN           NaN   NaN           NaN   
...                               ...           ...   ...           ...   
R24 2021-05-26 23:57:30  2.437221e+07  1.303292e+08  38.4  2.437221e+07   
    2021-05-26 23:58:00  2.439498e+07  1.304510e+08  36.5  2.439498e+07   
    2021-05-26 23:58:30  2.441775e+07  1.305727e+08  34.1  2.441775e+07   
    2021-05-26 23:59:00  2.444052e+07  1.306945e+08  37.2  2.444052e+07   
    2021-05-26 23:59:30  2.446328e+07  1.308162e+08  38.7  2.446328e+07   

                        

In [ ]:
# Definition of time range in which we want to carry out the analysis (get Satellite position, velocity and clockoffset and
# perform Point Positioning)
time_range = []

START = df_gps['time'].min()
END = dtt.strptime('2021-05-27 00:00:00', '%Y-%m-%d %H:%M:%S')
t = START
while t <= END:
    time_range.append(t)
    t = t + datetime.timedelta(minutes=5)
mask_par = df_gps['time'] <= END
df_gps = df_gps[mask_par].reset_index().drop(columns=['index'])
    
# List of the timestamps of the available observation (we can perform Point Positioning only when we have sufficient 
# code observations)
time_obs = obs.time.to_dataframe()['time'].tolist()

# Satellites positions, velocities and clock offsets

In [ ]:
# Definition of the empty DataFrame that will contain the results
satellites = pd.DataFrame(columns = ['time', 'sv', 'xs', 'ys', 'zs', 'xs_dot', 'ys_dot', 'zs_dot', 'ts', 'C1'])

for w in time_range:
    # The list of satellites, whose parameter we want to estimate, is obtained from the available observations:
    # for each instant of time in time_range we select the in-view satellite
    # their position, velocity and clock offset are the computed
    if w in time_obs:
        obs_tk = (obs.sel(time = w)).to_dataframe().reset_index()
        obs_tk = obs_tk[['time', 'sv', 'C1']].dropna().reset_index()
        available_sat = (obs_tk['sv']).to_list()
        if len(available_sat) > 0:
            for sv_i in available_sat:
                # Every satellite in view at the considered epoch is associated with the parameter slot 
                # to which it belongs through the getPar function
                sat_par = fn.getPar(sv_i, w, df_gps)
                # If the navigation parameters are available:
                if type(sat_par) == pd.core.frame.DataFrame:
                    sat = fn.satPosVel(sat_par).reset_index()
                    sat = sat.merge(obs_tk, on = ['sv', 'time'], how = 'left')
                    satellites = satellites.append(sat)
        
satellites = satellites.reset_index()
satellites = satellites[['time', 'sv', 'xs', 'ys', 'zs', 'xs_dot', 'ys_dot', 'zs_dot', 'ts', 'C1']]

print(satellites)

In [ ]:
satellites['P1'] = satellites['C1']

# Check on satellites' position and clock offsets

In [ ]:
confronto = fn.checkSatPos(satellites, eph_path)

# Check on satellites' velocity

In [ ]:
velocita = fn.checkSatVel(satellites, df_gps)

# Point Positioning

In [ ]:
# Alpha and Beta paramethers for computing Ionosphere correction are read from nav RINEX through an apposite function
ionoParams = RINEXreader.getIonoParams(nav_path)
print('Inosphere parameters: ', ionoParams)

# Definition of the initial position of the receiver (from OBS RINEX)
R_0 = RINEXreader.getStartPos(obs_path)

# Performing of Point Positioning with the apposite function
results = pp.pointPositioning(satellites, R_0, ionoParams, cutoff)

print(results)

In [ ]:
print('Statistical analysis on Point Positioning results:')
print('Mean values: ')
print('XR: ', results['xr'].mean())
print('YR: ', results['yr'].mean())
print('ZR: ', results['zr'].mean())
print('dtR: ', results['dtr'].mean())

print('Standard deviation: ')
print('XR: ', results['xr'].std())
print('YR: ', results['yr'].std())
print('ZR: ', results['zr'].std())
print('dtR: ', results['dtr'].std())

# Transformation in Local Cartesian

In [ ]:
# Coordinates of the receiver, Milan Permanent Station, reference: SPIN3GNSS
#P0_g = [45.478368630552694, 9.229212872263435, 191.12520962953568]
#P0_cart = trf.geodToCart(45.478368630552694, 9.229212872263435, 191.12520962953568)

#Zimmerwald
P0_cart = [4331300.16, 567537.08, 4633133.51]
P0_g = trf.cartToGeod(4331300.16, 567537.08, 4633133.51)

results_LC = pd.DataFrame()
results_LC['dx'] = results['xr'] - P0_cart[0]
results_LC['dy'] = results['yr'] - P0_cart[1]
results_LC['dz'] = results['zr'] - P0_cart[2]

E=[]
N=[]
U=[]

lat0 = trf.degToRad(P0_g[0])
lon0 = trf.degToRad(P0_g[1])

# Definition of the rotation matrix
R = np.array([ [-np.sin(lon0), np.cos(lon0), 0],
                [-np.sin(lat0)*np.cos(lon0), -np.sin(lat0)*np.sin(lon0), np.cos(lat0)],
                [np.cos(lat0)*np.cos(lon0), np.cos(lat0)*np.sin(lon0), np.sin(lat0)]
                ])

for i in range(len(results_LC)):
    delta_array_i = np.array([[results_LC['dx'][i]], [results_LC['dy'][i]], [results_LC['dz'][i]]])
    ENU =np.dot(R, delta_array_i)
    E.append(ENU[0][0])
    N.append(ENU[1][0])
    U.append(ENU[2][0])

results_LC['E'] = E
results_LC['N'] = N
results_LC['U'] = U
results_LC['time'] = results['datetime']
results_LC = results_LC.reset_index()

In [ ]:
print('MEDIA ERRORI:')
print('Est: ', results_LC['E'].mean(), '\n',
      'Nord: ', results_LC['N'].mean(), '\n',
      'Quota: ', results_LC['U'].mean() )

print('DEVIAZIONE STANDARD:', '\n',
      'Est: ', results_LC['E'].std(), '\n',
      'Nord: ', results_LC['N'].std(), '\n',
      'Quota: ', results_LC['U'].std() )

print('Valori Massimi:', '\n',
      'Est: ', abs(results_LC['E']).max(), '\n',
      'Nord: ', abs(results_LC['N']).max(), '\n',
      'Quota: ', abs(results_LC['U']).max() )

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(results_LC['time'], results_LC['E'],
        '-',
        color='red')
ax.set(xlabel="Time", ylabel='EAST', title = 'Time-EAST')
ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_major_formatter(DateFormatter("%H-%M"))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(results_LC['time'], results_LC['N'],
        '-',
        color='blue')
ax.set(xlabel="Time", ylabel='NORTH', title = 'Time-NORTH')
ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_major_formatter(DateFormatter("%H-%M"))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(results_LC['time'], results_LC['U'],
        '-',
        color='green')
ax.set(xlabel="Time", ylabel='UP', title = 'Time-UP')
ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_major_formatter(DateFormatter("%H-%M"))
plt.show()